# LMS Exploration | 1. Rock_Scissor_Paper Classifier
---
**[Introduce]** 
- 가위, 바위, 보 이미지 분류기
- TensorFlow의 표준 API인 tf.keras의 Sequential APi를 이용하여 LeNet 을 바탕으로 딥러닝 네트워크를 설계한다.


- Image Classifier of rock, scissor, paper
- Design DeepLearning Network using LeNet


**[dataset]** 
- 10명 이상의 다양한 손가락 이미지를 사용하여 가위, 바위, 보 각각 약 1400개의 data를 이용하여 훈련했으며, train dataset에 포함되지 않은 약 300 개의 test data로 테스트 했다.

- 가위, 바위, 보 3개의 class 각각 약 1400개의 train data (총 4290 개)
- 가위, 바위, 보 3개의 class 각각 약 100개의 test data (총 300 개)


**[HyperParameter]** 
- 최적의 HyperParameter를 구하기 위해 검증 함수를 사용했다.


**[Preparation]** 
- mkdir -p : 디렉토리 만들기 (make directory)
- unzip : 이미지 압축 해제 (unzip zip files)

## 1. Preparation

- make directory (아래 폴더구조 참고)
- unzip zip files


---

**rock_scissor_paper**

     #- train dataset
     -rock
     -scissor
     -paper
     -test
         #- test dataset
         -rock
         -scissor
         -paper
         
--- 


## 2. 라이브러리 불러오기 | import library

In [3]:
from PIL import Image
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## 3. 이미지 크기 변환하기 | resize images
244x244 크기의 이미지를 28x28 크기로 변환한다.

In [11]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
	print(len(images), " images to be resized.")
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [5]:
dir = ["scissor", "rock", "paper", "test/rock", "test/scissor", "test/paper"]

for i in dir:
    image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/"+i
    resize_images(image_dir_path)

1530  images to be resized.
1530  images resized.
1536  images to be resized.
1536  images resized.
1528  images to be resized.
1528  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.
100  images to be resized.
100  images resized.


## 4. 훈련 및 테스트 데이터 불러오기 | load train & test dataset

In [6]:
def load_data(img_path, number_of_data):
    #- 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #- 이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    #- 데이터 영역에 이미지 행렬 복사
        labels[idx]=0   
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=1  
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    
        labels[idx]=2  
        idx=idx+1

    return imgs, labels

In [7]:
def number_of_data(a):
    dir = ["scissor", "rock", "paper"]
    number=0
    if a == 'train':
        for i in dir:
            image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/"+i
            number += len(os.listdir(image_dir_path))
        print("학습데이터(x_train)의 이미지 개수는", number,"입니다.")
    if a == 'test':
        for i in dir:
            image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/"+i
            number += len(os.listdir(image_dir_path))
        print("테스트데이터(x_test)의 이미지 개수는", number,"입니다.")
    return number


image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
x_train, y_train=load_data(image_dir_path, number_of_data('train'))


image_dir_path = image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
x_test, y_test= load_data(image_dir_path, number_of_data('test'))

학습데이터(x_train)의 이미지 개수는 4601 입니다.
테스트데이터(x_test)의 이미지 개수는 306 입니다.


## 5. 데이터 정규화 | normalize dataset

In [8]:
x_train_norm = x_train/255.0
x_test_norm = x_test/255.0

## 6. 하이퍼파라미터 검증 | evaluate HyperParameter
다양한 하이퍼파라미터 조합을 검증함으로써 최적의 test accuracy 값을 도출해낸다.   

**[Best Hyperparameter for this model]**
    
channel_1 : 128   
channel_2 : 256    
dense : 64       
best_test_accuracy: 0.8627451062202454   

In [9]:
from itertools import product

n_train_epoch=10

def get_model(n_channel_1, n_channel_2, n_dense):
    for channel_1, channel_2, dense in product(n_channel_1, n_channel_2, n_dense):
        model=keras.models.Sequential()
        model.add(keras.layers.Conv2D(channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
        model.add(keras.layers.MaxPool2D(2,2))
        model.add(keras.layers.Conv2D(channel_2, (3,3), activation='relu'))
        model.add(keras.layers.MaxPooling2D((2,2)))
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(dense, activation='relu'))
        model.add(keras.layers.Dense(3, activation='softmax'))

        model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
        yield model, [channel_1, channel_2, dense]

n_channel_1 = [16, 32, 64, 128]
n_channel_2 = [32, 64, 128, 256]
n_dense = [32, 64, 128, 256]

best_test_acc = 0
best_args = []

for model, args in get_model(n_channel_1, n_channel_2, n_dense):
    model.fit(x_train, y_train, epochs=n_train_epoch)

    test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
    print(f"test_loss: {test_loss} ")
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        best_args = args
        print("BEST!!!")
    print(f"channel_1 : {args[0]} channel_2 : {args[1]} dense : {args[2]} test_accuracy: {test_accuracy}\n\n")


print(f"<Best Hyperparameter for this model> \nchannel_1 : {best_args[0]}\nchannel_2 : {best_args[1]}\ndense : {best_args[2]}\nbest_test_accuracy: {best_test_acc}")

Epoch 1/10
144/144 [==============================] - 5s 15ms/step - loss: 9.4748 - accuracy: 0.4063
Epoch 2/10
144/144 [==============================] - 0s 3ms/step - loss: 1.0780 - accuracy: 0.6301
Epoch 3/10
144/144 [==============================] - 0s 3ms/step - loss: 0.5788 - accuracy: 0.7724
Epoch 4/10
144/144 [==============================] - 0s 3ms/step - loss: 0.4191 - accuracy: 0.8289
Epoch 5/10
144/144 [==============================] - 0s 3ms/step - loss: 0.2947 - accuracy: 0.8840
Epoch 6/10
144/144 [==============================] - 0s 3ms/step - loss: 0.2368 - accuracy: 0.9064
Epoch 7/10
144/144 [==============================] - 0s 3ms/step - loss: 0.1527 - accuracy: 0.9451
Epoch 8/10
144/144 [==============================] - 0s 3ms/step - loss: 0.1532 - accuracy: 0.9503
Epoch 9/10
144/144 [==============================] - 0s 3ms/step - loss: 0.1027 - accuracy: 0.9630
Epoch 10/10
144/144 [==============================] - 0s 3ms/step - loss: 0.0699 - accuracy: 0.977

## 7. 딥러닝 네트워크 설계, 학습, 테스트 | Design, Train, Test DeepLearning Network 

- Lenet 바탕의 DeepLearning Network
- fit() : 학습
- evaluate() : 테스트

In [10]:
#- 딥러닝네트워크 설계
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

#- 학습
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=10)

#- 테스트
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))


Model: "sequential_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_128 (Conv2D)          (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_128 (MaxPoolin (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 11, 11, 128)       36992     
_________________________________________________________________
max_pooling2d_129 (MaxPoolin (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_64 (Flatten)         (None, 3200)              0         
_________________________________________________________________
dense_128 (Dense)            (None, 64)                204864    
_________________________________________________________________
dense_129 (Dense)            (None, 3)               

## 8. 프로젝트 정리 | Summary
---

### 8-1. 최종결과

**train dataset (개수, 다양성)** : 4601개, 10명의 dataset을 활용했으며 각 클래스별 균등한 개수의 데이터를 할당했다. 

**test dataset (개수, 다양성)** : 306개, 1명의 dataset을 활용했으며 각 클래스별 균등한 개수의 데이터를 할당했다. 

**channel1, channel2, dense** = 128 , 256 , 64

**test_loss, test_accuracy** = 1.0453828573226929 , 0.8627451062202454


### 8-2. 과정

**첫번째 시도**에서, 본인의 손가락 이미지 데이터 300개로 훈련시키고, 다른 한 명의 손가락 이미지 300개로 테스트했다. 그 결과, accuracy 가 1.0에 수렴했다. 원인분석 결과, 데이터로드 중 실수로 테스트 이미지에 본인의 이미지가 다수 들어있어 과적합이 발생했다.

**두번째 시도**에서, 10명의 이미지 데이터 총 4000개로 훈련시키고, 훈련데이터와 중복되지 않는 다른 한 명의 손가락 이미지 300개로 테스트했다. 그 결과, accuracy가 0.6을 겨우 넘겼다. 원인분석 결과 하이퍼파라미터의 조절이 필요하다고 생각되어, 검증함수를 도입했고, 다양성에 주의를 기울여 약 600개의 훈련 데이터를 추가했다.

**세번째 시도**에서, 11명의 이미지 데이터 총 4600개로 훈련시키고, 훈련데이터와 중복되지 않는 다른 한 명의 손가락 이미지 300개로 테스트했다. 그 결과 test accuracy 약 0.86 의 결과를 얻었다. 

### 8-3. 자가평가 

DeepLearning Model의 학습에서 Model도 중요하지만 무엇보다도 **dataset의 질**이 중요하다는 것을 느꼈다. 지도학습 분류모델에서, overfitting을 방지하기 위해서는 각 클래스의 데이터가 균형잡혀야 하며, 다양한 데이터가 필요하다는 것을 배웠다. 앞으로 딥러닝모델을 학습시킬 때 있어서 데이터의 정제에 특별히 주의를 기울여야겠다.     
